<a href="https://colab.research.google.com/github/Bunkhuoch-Ann/Astro_session/blob/main/Coding_session2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
from astropy.io import fits
import matplotlib.pyplot as plt
import numpy as np
from photutils.detection import DAOStarFinder
from astropy.stats import mad_std

In [40]:
# !pip install photutils

In [ ]:
from PIL import Image
import requests
import numpy as np
import matplotlib.pyplot as plt

# Load the RGB image from the URL
url = 'https://stsci-opo.org/STScI-01G8H1NK4W8CJYHF2DDFD1W0DQ.png'
img = Image.open(requests.get(url, stream=True).raw).convert("RGB")

# Convert to NumPy array
rgb = np.array(img)  # shape: (height, width, 3)

# Show the image
plt.figure(figsize=(8, 8))
plt.imshow(rgb, origin='lower')
plt.title("Hubble Deep Field (RGB)")
plt.axis('off')
plt.show()


In [ ]:
from PIL import Image
import requests

# Load the RGB image from URL or file
url = 'https://stsci-opo.org/STScI-01G8H1NK4W8CJYHF2DDFD1W0DQ.png'
img = Image.open(requests.get(url, stream=True).raw)

# Convert to grayscale
gray = img.convert('L')
data = np.array(gray)

# Show the image
plt.figure(figsize=(8, 8))
plt.imshow(data, cmap='gray', origin='lower')
plt.title("Converted Grayscale of Hubble Deep Field")
plt.colorbar()
plt.show()

In [ ]:
from photutils.detection import DAOStarFinder
from astropy.stats import sigma_clipped_stats

mean, median, std = sigma_clipped_stats(data, sigma=3.0)
print(f"Estimated background: median = {median:.2f}, std = {std:.2f}")

In [ ]:
from photutils.detection import IRAFStarFinder
from astropy.stats import sigma_clipped_stats

# Estimate the background
mean, median, std = sigma_clipped_stats(data, sigma=3.0)

# Use IRAFStarFinder to get sharpness, roundness, and FWHM estimates
iraf = IRAFStarFinder(threshold=5*std, fwhm=10.0)  # initial guess for fwhm
sources = iraf(data - median)

# View FWHMs of detected sources
fwhms = sources['fwhm']
print(f"Median FWHM: {np.median(fwhms):.2f} pixels")

In [ ]:
# === Step 4: Detect sources ===
daofind = DAOStarFinder(fwhm=5.76, threshold=3.0 * std)
sources = daofind(data - median)
print(f"✅ Found {len(sources)} sources")

In [49]:
from photutils.aperture import CircularAperture
# === Step 5: Plot with circular apertures ===
positions = np.transpose((sources['xcentroid'], sources['ycentroid']))
apertures = CircularAperture(positions, r=3.0)

In [ ]:
plt.figure(figsize=(10, 8))
plt.imshow(data, cmap='gray', origin='lower')
apertures.plot(color='red', lw=1.0)
plt.title("Detected Sources in RGB Image")
plt.axis('off')
plt.show()

In [ ]:
data.shape

At this point we get the number of sources in a given field of view of the sky. The FoV of the sky in this image is said to be around 2.4 arcmin, approximately equal to a grain of sand held at arm's length. So there are thousands of galaxies in this tiny region!

In [ ]:
FoV = 2.4 #arcminutes
Fov_deg = FoV/60
print(f'The FoV of the image is {FoV} arcmin or {Fov_deg} degrees')

In [ ]:
#We want our data to be in number of galaxy/sterradian so that is why we need to convert the deg to str
Fov_rad = Fov_deg*np.pi/180
print(f'The FoV of the image is {Fov_rad} radians')

In [ ]:
#The area of the sky in str that this image cover is
Area_str = Fov_rad**2
print(f'The area of the sky that this image covers is {Area_str} steradian')

In [ ]:
#Galaxy number density per steradian is
print(f"There are {len(sources)} in {Area_str} steradian of the sky")
print(f"number density is: {len(sources)} / {Area_str} str")
density = len(sources)/Area_str
print(f'or {density} gal/str')

In [ ]:
#There are 4pi str for the entire sky, therefore the number of the galaxies in the observable universe is:
print(f'The number of galaxies in the observable universe is {density*4*np.pi} gal')

In [ ]:
density*4*np.pi*(10**(-12))

**Note**: This estimate is about 20 times lower than the commonly quoted value of 2 trillion galaxies. However, we should still be proud — we've reached this ballpark estimate using just a simple method and not advanced source detection algorithms.

### **Try accessing the Fits file**

In [ ]:
from astropy.io import fits
import requests
from io import BytesIO

url = "https://raw.githubusercontent.com/Bunkhuoch-Ann/Astro_session/main/cropped_deepfield.fits"

# Load FITS file directly from GitHub
response = requests.get(url)
hdul = fits.open(BytesIO(response.content))

# Access data and header
data = hdul[0].data
header = hdul[0].header

# Show basic info
print("Shape:", data.shape)
hdul.info()

In [67]:
# Replace NaNs and negatives with small positive value
crop = np.nan_to_num(data, nan=0.0)
crop[crop <= 0] = np.min(crop[crop > 0])  # smallest positive value
log_crop = np.log10(crop)

In [ ]:
vmin, vmax = np.percentile(log_crop, [5, 99])  # stretch between 5th and 99th percentile

plt.figure(figsize=(10, 8))
plt.imshow(log_crop, cmap='gray', origin='lower', vmin=vmin, vmax=vmax)
plt.colorbar(label='log10(flux)')
plt.title("Log10-Stretched Deep Field Crop")
plt.axis('off')
plt.show()

In [ ]:
header

In [ ]:
from photutils.detection import DAOStarFinder
from astropy.stats import sigma_clipped_stats

# === Step 2: Basic preprocessing ===
data = np.nan_to_num(data, nan=0.0)
# data[data < 0] = 0  # Optional: remove negatives

# === Step 3: Estimate background statistics ===
mean, median, std = sigma_clipped_stats(data, sigma=3.0)
print(f"Background mean = {mean:.3f}, std = {std:.3f}")

In [71]:
from photutils.detection import DAOStarFinder
from astropy.stats import sigma_clipped_stats
import numpy as np

# Estimate background
mean, median, std = sigma_clipped_stats(data, sigma=3.0)

# Detect sources (with high threshold to find just bright stuff)
daofind = DAOStarFinder(fwhm=3.0, threshold=10.0 * std)
sources = daofind(data - median)

# Find the brightest one
brightest = sources[np.argmax(sources['peak'])]

x0, y0 = brightest['xcentroid'], brightest['ycentroid']
r = 30 # mask radius in pixels

# Create circular mask
yy, xx = np.indices(data.shape)
mask = ((xx - x0)**2 + (yy - y0)**2) < r**2

# Apply mask
masked_data = data.copy()
masked_data[mask] = np.nan


In [ ]:
from photutils.detection import IRAFStarFinder
from astropy.stats import sigma_clipped_stats

# Estimate the background
mean, median, std = sigma_clipped_stats(masked_data, sigma=3.0)

# Use IRAFStarFinder to get sharpness, roundness, and FWHM estimates
iraf = IRAFStarFinder(threshold=5*std, fwhm=10.0)  # initial guess for fwhm
sources = iraf(masked_data - median)

# View FWHMs of detected sources
fwhms = sources['fwhm']
print(f"Median FWHM: {np.median(fwhms):.2f} pixels")

In [ ]:
# === Step 4: Detect sources ===
# threshold = median + 5*std is a good start for deep fields
mean, median, std = sigma_clipped_stats(masked_data, sigma=3.0)
sources = DAOStarFinder(fwhm=10, threshold=3.0 * std)(masked_data - median)

print(f"✅ Found {len(sources)} sources")


In [ ]:
from photutils.aperture import EllipticalAperture

# === Step 5: Draw elliptical apertures instead of scatter dots ===

# Coordinates of the detections
positions = np.transpose((sources['xcentroid'], sources['ycentroid']))

# Create elliptical apertures (a = b = 3 px, theta = 0)
apertures = EllipticalAperture(positions, a=3, b=3, theta=0.0)

# Plot
plt.figure(figsize=(10, 8))
plt.imshow(log_crop, cmap='gray', origin='lower', vmin=vmin, vmax=vmax)
apertures.plot(color='lime', lw=1.0)
plt.title("Detected Sources with Elliptical Apertures")
plt.xlabel("X Pixel")
plt.ylabel("Y Pixel")
plt.show()

In [ ]:
# Area of the sky in str per pixel of this image
Area = header['PIXAR_A2'] #arcsecond^2
print(f'The area of the sky that this image covers is {Area} arcsecond^2')

In [ ]:
# How many degree in the sky covers by 1 pixel
Pixel_scale = np.sqrt(Area)/3600 #degree
print(f'The area of the sky that this image covers is {Pixel_scale} degree')

**Assignment** fill in the blank. You can ask us for help!

In [ ]:
# Convert the pixel scale from degree in to rad
Pixel_scale_rad =
print(f'The area of the sky that this image covers is {Pixel_scale_rad} radians')

In [ ]:
# Shape of this image
shape = masked_data.shape
print(f'The shape of the image is {shape}')

In [ ]:
#Number of pixel in this image
N_pixel =
print(f'The number of pixel in this image is {N_pixel}')

In [ ]:
#Area of the sky cover by one pixel
Area_pixel =
print(f'The area of the sky that this image covers is {Area_pixel} steradian')

In [ ]:
#Area of the sky cover by this image
Area_image =
print(f'The area of the sky that this image covers is {Area_image} steradian')

In [ ]:
#Now calculate the number of galaxy per steradian
#Galaxy number density per steradian is
print(f"There are {len(sources)} in {Area_image} steradian of the sky")
print(f"number density is: {len(sources)} / {Area_image} str")
density =
print(f'or {density} gal/str')

In [ ]:
# Now calculate the number of galaxy in the observable universe
#There are 4pi str for the entire sky, therefore the number of the galaxies in the observable universe is:
print(f'The number of galaxies in the observable universe is {} gal')

Does the result agree with the previous image?